Important imports

In [1]:
import torch
import torchvision
from pathlib import Path
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim


How to unzip a zip file that contains the training data.

In [3]:
import os
import requests
import zipfile
from pathlib import Path

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "face"

# If the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)


# Unzip
with zipfile.ZipFile(data_path / "face.zip", "r") as zip_ref:
    print("Unzipping data...")
    zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "face.zip")



data/face directory exists.
Unzipping data...


How to transform the datapoints(images), create datasets and dataloaders.

In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

batch_size = 64
train_dir = Path("data/face/train/")
test_dir = Path("data/face/test/")
IMG_SIZE = 224
# Create transform pipeline manually

transform = transforms.Compose([
    transforms.Resize((224, 224)),      # Resize to 224x224
    transforms.ToTensor(),              # Convert to tensor [0,1]
    transforms.RandomRotation(9),
    transforms.Normalize(               # Normalize using ImageNet stats
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Use ImageFolder to create dataset(s)
train_data = datasets.ImageFolder(train_dir, transform=transform)
test_data = datasets.ImageFolder(test_dir, transform=transform)

# Get class names
class_names = train_data.classes

# Turn images into data loaders
train_loader = DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True,
    pin_memory=True,
)
test_loader = DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False, # don't need to shuffle test data
    pin_memory=True,)

How to load a pretrained ViT (Vision Transformer model) from torchvision.models, freeze all the parameters so that the training doesn't affect them, and replace the classification head so that it output matches the number of classes of that specific problem.


In [15]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Load the pretrained ViT model
model = models.vit_b_16(pretrained=True)

# Freeze the feature extractor
for param in model.parameters():
    param.requires_grad = False

# Replace the classification head for 7 classes
model.heads = nn.Sequential(
    nn.Linear(model.hidden_dim, 7)
)

model.to(device)

# Loss function and optimizer (only train classifier head)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.heads.parameters(), lr=1e-3)



cuda


The training loop

In [16]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / total
    accuracy = correct / total * 100
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")



Epoch 1/10, Loss: 1.2538, Accuracy: 52.76%
Epoch 2/10, Loss: 1.0590, Accuracy: 61.01%
Epoch 3/10, Loss: 0.9905, Accuracy: 63.79%
Epoch 4/10, Loss: 0.9485, Accuracy: 65.22%
Epoch 5/10, Loss: 0.9177, Accuracy: 66.21%
Epoch 6/10, Loss: 0.8998, Accuracy: 66.89%
Epoch 7/10, Loss: 0.8776, Accuracy: 68.08%
Epoch 8/10, Loss: 0.8598, Accuracy: 68.68%
Epoch 9/10, Loss: 0.8566, Accuracy: 68.95%
Epoch 10/10, Loss: 0.8352, Accuracy: 69.28%


How to put the model into evaluation mode, use torch.no_grad() and calculate the validation/test accuracy by using data from from the test dataloader.

In [7]:
# (Optional) Validation loop
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

val_accuracy = correct / total * 100
print(f"Validation Accuracy: {val_accuracy:.2f}%")

Validation Accuracy: 67.46%


How to load a pretrained VGG model from torchvision.models, freeze all the parameters so that the training doesn't affect them, and replace the classification head so that it output matches the number of classes of that specific problem.

In [8]:
model = models.vgg11(pretrained=True)

# Freeze all feature extractor layers
for param in model.features.parameters():
    param.requires_grad = False

# Replace the classifier for 7 classes
model.classifier = nn.Sequential(
    nn.Linear(25088, 4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 7)  # 7 classes
)
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

# Loss and optimizer (train only classifier)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=1e-3)



/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to /root/.cache/torch/hub/checkpoints/vgg11-8a719046.pth


100%|██████████| 507M/507M [00:03<00:00, 175MB/s]


cuda


The training loop

In [ ]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / total
    accuracy = correct / total * 100
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")



Epoch 1/10, Loss: 1.2445, Accuracy: 56.95%
Epoch 2/10, Loss: 0.8708, Accuracy: 68.86%
Epoch 3/10, Loss: 0.7743, Accuracy: 72.80%
Epoch 4/10, Loss: 0.6912, Accuracy: 75.59%
Epoch 5/10, Loss: 0.6377, Accuracy: 77.61%


How to save the weights of a model via a .pth (.pt) file

In [10]:
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': epoch_loss,
}, 'vgg11_checkpoint.pth')

How to load the weights of a model which has been saved in a .pth file.

In [11]:
# Load model and optimizer
checkpoint = torch.load('vgg11_checkpoint.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Optional: resume from the next epoch
start_epoch = checkpoint['epoch'] + 1

How to put the model into evaluation mode, use torch.no_grad() and calculate the validation/test accuracy by using data from from the test dataloader.

In [12]:
# Optional: validation loop
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

val_accuracy = correct / total * 100
print(f"Validation Accuracy: {val_accuracy:.2f}%")

Validation Accuracy: 73.97%


How to adjust the learning rate of the optimizer

In [13]:
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.5  # or 0.1

How to refactor the code block that calculates the test accuracy of a model into a function whose inputs are the model, the test dataloader and the device on which the model is.

In [ ]:
def evaluate_accuracy(model, test_loader, device):
    model.eval()  # set model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # disable gradient computation
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy


In [ ]:
print(evaluate_accuracy(model, test_loader, device))

Test Accuracy: 61.40%
61.395932014488714


How to install and use the summary tool from torchinfo to get a summary of a models architecture, the input and output shapes of each layer and the number of parameters in each layer.

In [ ]:
%pip install torchinfo
import torchinfo
from torchinfo import summary

# Print a summary of our custom ViT model using torchinfo (uncomment for actual output)
summary(model=model,
        input_size=(32, 3, 224, 224), # (batch_size, color_channels, height, width)
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
VGG (VGG)                                [32, 3, 224, 224]    [32, 7]              --                   Partial
├─Sequential (features)                  [32, 3, 224, 224]    [32, 512, 7, 7]      --                   False
│    └─Conv2d (0)                        [32, 3, 224, 224]    [32, 64, 224, 224]   (1,792)              False
│    └─ReLU (1)                          [32, 64, 224, 224]   [32, 64, 224, 224]   --                   --
│    └─MaxPool2d (2)                     [32, 64, 224, 224]   [32, 64, 112, 112]   --                   --
│    └─Conv2d (3)                        [32, 64, 112, 112]   [32, 128, 112, 112]  (73,856)             False
│    └─ReLU (4)                          [32, 128, 112, 112]  [32, 128, 112, 112]  --                   --
│    └─MaxPool2d (5)                     [32, 128, 112, 112]  [32, 128, 56, 56]    --                   --
│    └─Conv2d (6